In [12]:
# ## Install Coqui TTS
# ! pip install -U pip
# ! pip install TTS==0.8.0
# import os
# !git clone https://github.com/coqui-ai/TTS
# os.chdir('TTS/')

# !python3 -m pip install -r requirements.txt
# !python3 setup.py develop

# !ls


In [13]:
!nvidia-smi
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import torch
# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.vits import Vits, VitsAudioConfig
# BaseDatasetConfig: defines name, formatter and path of the dataset.


from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

from TTS.tts.configs.shared_configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig
#from TTS.configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig#GlowTTSConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.speakers import SpeakerManager

from TTS.config import load_config

from trainer import Trainer, TrainerArgs


Fri Aug 11 12:17:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:08:00.0  On |                  Off |
| 31%   46C    P8    28W / 480W |    454MiB / 24248MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
# # !apt install -y espeak
# !apt install -y espeak-ng
!pwd

/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts


In [15]:
# !ls bn_tts

In [16]:
import TTS
TTS.__version__

'0.14.3'

In [17]:

# output_path = "tts_train_dir"
# if not os.path.exists(output_path):
#     os.makedirs(output_path)

male = False
pretrained = False

pretrained_path = ''
if(pretrained):
    pretrained_path = ''
if(male):
    meta_file = '/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/iitm_bangla_tts/comprehensive_bangla_tts/male/mono/metadata_male.txt'
    root_path = '/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/iitm_bangla_tts/comprehensive_bangla_tts/male/mono'
else:
    meta_file = '/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/iitm_bangla_tts/comprehensive_bangla_tts/female/mono/metadata_female.txt'
    root_path = '/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/iitm_bangla_tts/comprehensive_bangla_tts/female/mono'

def formatter(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the LJSpeech meta data file to TTS format
    https://keithito.com/LJ-Speech-Dataset/"""
    txt_file = meta_file
    items = []
    speaker_name = "ljspeech"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wav", cols[0] + ".wav")
            try:
                text = cols[1]
            except:
                print("not found")

            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


dataset_config = BaseDatasetConfig(
     meta_file_train=meta_file, 
    path=os.path.join(root_path, "")
)


In [18]:
dataset_config

BaseDatasetConfig(formatter='', dataset_name='', path='/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/iitm_bangla_tts/comprehensive_bangla_tts/female/mono/', meta_file_train='/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/iitm_bangla_tts/comprehensive_bangla_tts/female/mono/metadata_female.txt', ignored_speakers=None, language='', phonemizer='', meta_file_val='', meta_file_attn_mask='')

In [19]:
# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config,formatter=formatter, eval_split=True)

 | > Found 6665 files in /media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/iitm_bangla_tts/comprehensive_bangla_tts/female/mono


In [20]:
print(len(train_samples),len(eval_samples))

6599 66


In [21]:
train_samples[0]

{'text': 'দীর্ঘস্থায়ী অবসাদের ক্ষেত্রে, জীবনযাপনের প্রতিটি স্তরে, হতাশার লক্ষণ দেখা যায়\n',
 'audio_file': '/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/iitm_bangla_tts/comprehensive_bangla_tts/female/mono/wav/train_bengalifemale_01282.wav',
 'speaker_name': 'ljspeech',
 'root_path': '/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/iitm_bangla_tts/comprehensive_bangla_tts/female/mono/',
 'language': '',
 'audio_unique_name': '#wav/train_bengalifemale_01282'}

In [24]:
# we use the same path as this script as our training folder.
output_path = '/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/tts_model'

# audio_config = BaseAudioConfig(
#      sample_rate = 22050,
#     resample =True
# )

audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)


if(male):
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "তট৫ভিঐঋখঊড়ইজমএেঘঙসীঢ়হঞ‘ঈকণ৬ঁৗশঢঠ\u200c১্২৮দৃঔগও—ছউংবৈঝাযফ\u200dচরষঅৌৎথড়৪ধ০ুূ৩আঃপয়’নলো",
    punctuations = "-!,|.? ",
    )
else:
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "ইগং়’ুঃন১ঝূও‘ঊোছপফৈ৮ষযৎঢঈকঠিজ০৬ীটডএঅঋধচে২৩ণউয়ঢ়খলভৗসহ্ড়দথবঔাঞশরৌম—ঐআৃঘঙ\u200cঁ৪৫ত",
    punctuations = ".?-!|, ",
    )


# VitsConfig: all model related values for training, validating and testing.

config = VitsConfig(
    audio=audio_config,
    run_name="vits_4_nov",
    batch_size=64,
    eval_batch_size=32,
    batch_group_size=0,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=15000,
    text_cleaner=None, #"collapse_whitespace"
    use_phonemes=False,
    compute_input_seq_cache=True,
    print_step=50,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    characters = characters_config,
    save_step=1000,
    cudnn_benchmark=True,
    test_sentences = [
        'হয়,হয়ে,ওয়া,হয়েছ,হয়েছে,দিয়ে,যায়,দায়,নিশ্চয়,আয়,ভয়,নয়,আয়াত,নিয়ে,হয়েছে,দিয়েছ,রয়ে,রয়েছ,রয়েছে।',
        'দেয়,দেওয়া,বিষয়,হয়,হওয়া,সম্প্রদায়,সময়,হয়েছি,দিয়েছি,হয়,হয়েছিল,বিষয়ে,নয়,কিয়াম,ইয়া,দেয়া,দিয়েছে,আয়াতে,দয়া।', 
        'ইয়াহুদ,নয়,ব্যয়,ইয়াহুদী,নেওয়া,উভয়ে,যায়,হয়েছিল,প্রয়োজন।'
    ]
)


In [25]:

ap = AudioProcessor.init_from_config(config)
# ap.resample = True

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [26]:
ap.resample

False

In [27]:

tokenizer, config = TTSTokenizer.init_from_config(config)

In [28]:

model = Vits(config, ap, tokenizer, speaker_manager=None)

In [29]:
# checkpoint = torch.load('/home/ansary/Shabab/vits_20_october/best_model_311129.pth')
# model.load_state_dict(checkpoint["model"])

model

Vits(
  (text_encoder): TextEncoder(
    (emb): Embedding(86, 192)
    (encoder): RelativePositionTransformer(
      (dropout): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0): RelativePositionMultiHeadAttention(
          (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): RelativePositionMultiHeadAttention(
          (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (2): RelativePositionMultiHeadAttenti

In [30]:
# use continue_path argument below for retraining model from last saved weight https://github.com/coqui-ai/Trainer/blob/main/trainer/trainer.py#L210

trainer = Trainer(
    TrainerArgs(continue_path = pretrained_path), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

fatal: not a git repository (or any parent up to mount point /media/sayan)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /media/sayan)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 24
 | > Num. of Torch Threads: 12
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: True
 > Start Tensorboard: tensorboard --logdir=/media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/tts_model/vits_4_nov-August-11-2023_12+17PM-0000000

 > Model has 83050540 parameters


In [ ]:
%%time

trainer.fit()


 > EPOCH: 0/15000
 --> /media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/tts_model/vits_4_nov-August-11-2023_12+17PM-0000000

 > TRAINING (2023-08-11 12:17:44) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 6599
 | > Preprocessing samples
 | > Max text length: 226
 | > Min text length: 16
 | > Avg text length: 64.47492044249128
 | 
 | > Max audio length: 375706.0
 | > Min audio length: 12053.0
 | > Avg audio length: 119447.2292771632
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
জেলা প্রশাসন মারফত, তা রাষ্ট্রপতির কাছে পৌঁছে দেওয়া, হবে
ক্লার্ক আবার ম্যাথু হেডেনকে বিগ ফিশ, বলে ডাকে
তিন ক্ষেত্রেই পদক হাতছাড়া হচ্ছে ঘোড়া ডোপ করায়



 [!] Character '\n' not found in the vocabulary. Discarding it. [!] Character '\n' not found in the vocabulary. Discarding it. [!] Character '\n' not found in the vocabulary. Discarding it.


শুনলাম, মার্ভ হিউজও আছেন, এ দিক, ও দিক

 [!] Character '\n' not found in the vocabulary. Discarding it.



   --> STEP: 0/104 -- GLOBAL_STEP: 0
     | > loss_disc: 5.92577  (5.92577)
     | > loss_disc_real_0: 0.99313  (0.99313)
     | > loss_disc_real_1: 1.01208  (1.01208)
     | > loss_disc_real_2: 0.99774  (0.99774)
     | > loss_disc_real_3: 0.96984  (0.96984)
     | > loss_disc_real_4: 0.97330  (0.97330)
     | > loss_disc_real_5: 0.97907  (0.97907)
     | > loss_0: 5.92577  (5.92577)
     | > grad_norm_0: 0.00000  (0.00000)
     | > loss_gen: 5.92409  (5.92409)
     | > loss_kl: 139.59413  (139.59413)
     | > loss_feat: 0.32605  (0.32605)
     | > loss_mel: 82.67771  (82.67771)
     | > loss_duration: 3.15166  (3.15166)
     | > amp_scaler: 32768.00000  (32768.00000)
     | > loss_1: 231.67365  (231.67365)
     | > grad_norm_1: 0.00000  (0.00000)
     | > current_lr_0: 0.00020 
     | > current_lr_1: 0.00020 
     | > step_time: 6.40190  (6.40187)
     | > loader_time: 0.75960  (0.75961)


   --> STEP: 50/104 -- GLOBAL_STEP: 50
     | > loss_disc: 2.73258  (3.15230)
     | > loss_di



> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 66
 | > Preprocessing samples
 | > Max text length: 90
 | > Min text length: 41
 | > Avg text length: 64.28787878787878
 | 
 | > Max audio length: 194245.0
 | > Min audio length: 76489.0
 | > Avg audio length: 118537.27272727272
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
তাঁর অভিযোগ গোয়ালতোড় থানা এই ঘটনার অভিযোগ নেয়নি

ওই অঞ্চলে আছিক ন্যাশনাল ভল্যান্টিয়ার কাউন্সিল এএনভিসির প্রভাব রয়েছে

 [!] Character '\n' not found in the vocabulary. Discarding it. [!] Character '\n' not found in the vocabulary. Discarding it.

মহাভারতের ভীষ্মপর্বে অর্জুনও দেবীকে, হে কাত্যায়নী, মহাভাগে শাকম্ভরী বলে, সম্বোধন করেছিলেন

 [!] Character '\n' not found in the vocabulary. Discarding it.
 | > Synthesizing test sentences.


/home/sayan/anaconda3/envs/coqui/lib/python3.8/site-packages/TTS/tts/models/vits.py:1455: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2981.)
  test_figures["{}-alignment".format(idx)] = plot_alignment(alignment.T, output_fig=False)

  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.00783 (+0.00000)
     | > avg_loss_disc: 2.81658 (+0.00000)
     | > avg_loss_disc_real_0: 0.16410 (+0.00000)
     | > avg_loss_disc_real_1: 0.31201 (+0.00000)
     | > avg_loss_disc_real_2: 0.30429 (+0.00000)
     | > avg_loss_disc_real_3: 0.51956 (+0.00000)
     | > avg_loss_disc_real_4: 0.36779 (+0.00000)
     | > avg_loss_disc_real_5: 0.41047 (+0.00000)
     | > avg_loss_0: 2.81658 (+0.00000)
     

...


   --> STEP: 38/104 -- GLOBAL_STEP: 8150
     | > loss_disc: 2.65369  (2.63882)
     | > loss_disc_real_0: 0.11884  (0.20140)
     | > loss_disc_real_1: 0.16052  (0.23138)
     | > loss_disc_real_2: 0.27552  (0.23519)
     | > loss_disc_real_3: 0.22200  (0.23561)
     | > loss_disc_real_4: 0.23821  (0.24325)
     | > loss_disc_real_5: 0.25527  (0.24138)
     | > loss_0: 2.65369  (2.63882)
     | > grad_norm_0: 89.20673  (42.50074)
     | > loss_gen: 2.00957  (1.94196)
     | > loss_kl: 1.97996  (1.99952)
     | > loss_feat: 2.92359  (2.90791)
     | > loss_mel: 19.13914  (19.71442)
     | > loss_duration: 1.71052  (1.70512)
     | > amp_scaler: 256.00000  (256.00000)
     | > loss_1: 27.76278  (28.26894)
     | > grad_norm_1: 410.51834  (284.46494)
     | > current_lr_0: 0.00020 
     | > current_lr_1: 0.00020 
     | > step_time: 0.81740  (0.79998)
     | > loader_time: 0.00910  (0.00826)


   --> STEP: 88/104 -- GLOBAL_STEP: 8200
     | > loss_disc: 2.94543  (2.68931)
     | > loss_



> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 66
 | > Preprocessing samples
 | > Max text length: 90
 | > Min text length: 41
 | > Avg text length: 64.28787878787878
 | 
 | > Max audio length: 194245.0
 | > Min audio length: 76489.0
 | > Avg audio length: 118537.27272727272
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
তাঁর অভিযোগ গোয়ালতোড় থানা এই ঘটনার অভিযোগ নেয়নি
মহাভারতের ভীষ্মপর্বে অর্জুনও দেবীকে, হে কাত্যায়নী, মহাভাগে শাকম্ভরী বলে, সম্বোধন করেছিলেন

ওই অঞ্চলে আছিক ন্যাশনাল ভল্যান্টিয়ার কাউন্সিল এএনভিসির প্রভাব রয়েছে
 [!] Character '\n' not found in the vocabulary. Discarding it.

 [!] Character '\n' not found in the vocabulary. Discarding it.
 [!] Character '\n' not found in the vocabulary. Discarding it.

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.00401 (+0.00015)
     | > avg_loss_disc: 2.90070 (+0.04620)
     | > avg_loss_disc_real_0: 0.32345 (-0.02153)
     | > avg_loss_disc_real_1: 0.24971 (+0.03171)
     | > avg_loss_disc_real_2: 0.27419 (+0.04650)
     | > avg_loss_disc_real_3: 0.31522 (+0.00349)
     | > avg_loss_disc_real_4: 0.29169 (+0.00874)
     | > avg_loss_disc_real_5: 0.30330 (+0.01151)
     | > avg_loss_0: 2.90070 (+0.04620)
     | > avg_loss_gen: 2.01654 (+0.03875)
     | > avg_loss_kl: 2.36218 (+0.40097)
     | > avg_loss_feat: 2.19239 (-0.11213)
     | > avg_loss_mel: 21.16417 (+0.18888)
     | > avg_loss_duration: 1.86662 (+0.01091)
     | > avg_loss_1: 29.60191 (+0.52738)


 > EPOCH: 79/15000
 --> /media/sayan/hdd1/TTS/comprehensive-bangla-tts-main/bn_vits_tts/tts_model/vits_4_nov-August-11-2023_12+17PM-0000000

 > TRAINING (2023-08-11 14:21:34) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 6599
 | > Preprocessing samples
 | > Max text length: 226
 | > Min text length: 16
 | > Avg text length: 64.47492044249128
 | 
 | > Max audio length: 375706.0
 | > Min audio length: 12053.0
 | > Avg audio length: 119447.2292771632
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
জেলা প্রশাসন মারফত, তা রাষ্ট্রপতির কাছে পৌঁছে দেওয়া, হবে
ক্লার্ক আবার ম্যাথু হেডেনকে বিগ ফিশ, বলে ডাকে


তিন ক্ষেত্রেই পদক হাতছাড়া হচ্ছে ঘোড়া ডোপ করায়

 [!] Character '\n' not found in the vocabulary. Discarding it. [!] Character '\n' not found in the vocabulary. Discarding it.
 [!] Character '\n' not found in the vocabulary. Discarding it.

শুনলাম, মার্ভ হিউজও আছেন, এ দিক, ও দিক

 [!] Character '\n' not found in the vocabulary. Discarding it.



   --> STEP: 34/104 -- GLOBAL_STEP: 8250
     | > loss_disc: 2.72793  (2.65602)
     | > loss_disc_real_0: 0.31025  (0.20222)
     | > loss_disc_real_1: 0.26096  (0.23525)
     | > loss_disc_real_2: 0.24206  (0.23685)
     | > loss_disc_real_3: 0.23579  (0.23851)
     | > loss_disc_real_4: 0.27059  (0.24452)
     | > loss_disc_real_5: 0.26894  (0.23987)
     | > loss_0: 2.72793  (2.65602)
     | > grad_norm_0: 100.29988  (34.35117)
     | > loss_gen: 1.90555  (1.92445)
     | > loss_kl: 1.86224  (1.99033)
     | > loss_feat: 2.54733  (2.87090)
     | > loss_mel: 19.07455  (19.62464)
     | > loss_duration: 1.71640  (1.70102)
     | > amp_scaler: 256.00000  (256.00000)
     | > loss_1: 27.10607  (28.11135)
     | > grad_norm_1: 405.71600  (233.84183)
     | > current_lr_0: 0.00020 
     | > current_lr_1: 0.00020 
     | > step_time: 0.81900  (0.77943)
     | > loader_time: 0.00830  (0.00782)


   --> STEP: 84/104 -- GLOBAL_STEP: 8300
     | > loss_disc: 2.63123  (2.69032)
     | > loss

In [1]:
!nvidia-smi

Fri Aug 11 14:33:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:08:00.0  On |                  Off |
|  0%   43C    P8    27W / 480W |    511MiB / 24248MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------